In [2]:
using LinearAlgebra
using SparseArrays
using Arpack
using KrylovKit
using Dates
using JLD

## Specifying parameters of the model here:

In [1]:
L1=4;L2=4;J1=1;J2=1;h=0.1
N = L1*L2

16

## Building up Hamiltonian without using k space
- Building Hamiltonian for a generic rectangular lattice
- Using Int64 to represent states

Hamiltonian is $H=J\sum_{i=1}^{N}(S_i^z S_{i+1}^z + 1/2(S_i^+ S_{i+1}^- + S_i^- S_{i+1}^+)) $

## Building bond list (neib list) and input number of sites

### neib list for a rectangular lattice

Given the x and y dimension of the rectangular lattice (effectively, total number of sites $N$), we should output a neighbor list for all sites numbered from $1$ to $N$.

Here we are assuming we have a rectangular lattice of $L1$ in wide and $L2$ in length. So $L2$ rows and $L1$ columns in total.

In [3]:
function coordinate(n;L1::Int64=2,L2::Int64=2)
    @assert((n ≤ L1 * L2) && (1 ≤ n),"The numbering (bit position) of a site shouldn't exceed the total number of sites $(L1 * L2), and should be bigger than 0.")
    i::Int64 = Int(ceil(n/L1))
    j::Int64 = mod1(n,L1)  #site i is at i-th row, j-th column
    return (i,j)
end

function bit_pos(coordinate::Tuple{Int64,Int64};L1::Int64=2,L2::Int64=2)
    @assert((coordinate[1] ≤ L2) && (coordinate[2] ≤ L1),"The cooridnate should be within the range of the lattice size $L1 by $L2")
    n = (coordinate[1]-1)*L1 + coordinate[2]
    return n
end

bit_pos (generic function with 1 method)

### Generate neib list given that we know the dimension of the lattice

In [4]:
function nearest_neib(n::Int64;L1::Int64=2, L2::Int64=2)
    coord = coordinate(n,L1=L1, L2=L2)
    neibs = Tuple{Int64,Int64}[]
    push!(neibs, (mod1(coord[1]+1,L2), coord[2]))
    push!(neibs, (mod1(coord[1]-1,L2), coord[2]))
    push!(neibs, (coord[1], mod1(coord[2]+1,L1)))
    push!(neibs, (coord[1], mod1(coord[2]-1,L1)))    
    #=convert coordinations to positions in bits=#
    neibs_bit_pos = Set{Int64}()
    for neib in neibs
        push!(neibs_bit_pos, bit_pos(neib, L1=L1, L2=L2))
    end
    return neibs_bit_pos
end

function second_neib(n::Int64;L1::Int64=2, L2::Int64=2)
    coord = coordinate(n,L1=L1, L2=L2)
    neibs = Tuple{Int64,Int64}[]   
    if iseven(coord[1]+coord[2])
        push!(neibs, (mod1(coord[1]+1,L2), mod1(coord[2]-1,L1)))
        push!(neibs, (mod1(coord[1]-1,L2), mod1(coord[2]+1,L1)))
    else
        push!(neibs, (mod1(coord[1]+1,L2), mod1(coord[2]+1,L1)))
        push!(neibs, (mod1(coord[1]-1,L2), mod1(coord[2]-1,L1)))
    end
    #=convert coordinations to positions in bits=#
    neibs_bit_pos = Set{Int64}()
    for neib in neibs
        push!(neibs_bit_pos, bit_pos(neib, L1=L1, L2=L2))
    end
    return neibs_bit_pos
end

function nearest_neib_list_gen(;L1::Int64=2, L2::Int64=2)
    neib_list = Set{Int64}[]
    for n in 1:L1*L2
        push!(neib_list, nearest_neib(n, L1=L1, L2=L2))
    end
    return neib_list
end

function second_neib_list_gen(;L1::Int64=2, L2::Int64=2)
    neib_list = Set{Int64}[]
    for n in 1:L1*L2
        push!(neib_list, second_neib(n, L1=L1, L2=L2))
    end
    return neib_list
end

nearest_neib_list = nearest_neib_list_gen(L1=L1, L2=L2)
second_neib_list = second_neib_list_gen(L1=L1, L2=L2)

16-element Array{Set{Int64},1}:
 Set([14, 8])
 Set([7, 13])
 Set([16, 6])
 Set([5, 15])
 Set([4, 10])
 Set([9, 3]) 
 Set([2, 12])
 Set([11, 1])
 Set([16, 6])
 Set([5, 15])
 Set([14, 8])
 Set([7, 13])
 Set([2, 12])
 Set([11, 1])
 Set([4, 10])
 Set([9, 3]) 

## The Hamiltonian generator should intake bond list (neib list), number of sites and output the Hamiltonian:

So we need to revise the input of Hamiltonian function a little bit.

We still need the Hamiltonian in 2 different basis:

\begin{equation}
\tag{H1}
H=\sum_{i=1}^{N}\sum_{\sigma}JS_i^z S_{i+\sigma}^z + \frac{h}{2}\sum_{i=1}^{N} (S_i^+ + S_i^-)
\end{equation}

\begin{equation}
\tag{H2}
H=\sum_{i=1}^{N}\sum_{\sigma}\frac{J}{4}(S_i^+ + S_i^-)(S_{i+\sigma}^+ + S_{i+\sigma}^-) - h\sum_{i=1}^{N} S_i^z
\end{equation}

## Now we could construct Hamiltonian for even and odd sectors for H2 only
### First Let's construct Parity States

We can do a little rotation of our system and construct Hamiltonian in that "direction". The Hamiltonian should be:
$H=J\sum_{i=1}^{N}\sum_{\sigma}(S_i^x S_{i+\sigma}^x - h S_i^z)$

Now we have a nice Parity symmetry comes in handy. The Operator is $\prod_i \sigma_i^z$. The $\sigma_i^z$ basis is exactly the eigenbasis of the parity operator. The parity operator commutes with H, thus divides the Hamiltonian into 2 block diagonal parts. With one part having even total spin, one part having odd total spin. 

Physically we can see this by making use of the fact that $S_i^x S_{i+\sigma}^x$ flips 2 spins at a time. So different sectors can't mix.

First we need to loop over all states, and construct a list of even and odd states.

In [5]:
function chk_parity(state::Int64)
    state_binary = digits!(zeros(Int64, 64), state, base = 2)
    if iseven(sum(state_binary))
        return :even
    else
        return :odd
    end
end

function parity_states_list(;N::Int64, L1::Int64=L1, L2::Int64=L2)
    even_state = Int64[]
    odd_state = Int64[]
    even_state_num = Dict{Int64, Int64}()
    odd_state_num = Dict{Int64, Int64}()
    even_state_tot = 0
    odd_state_tot = 0
    for state in 0:(2^N-1)
        if chk_parity(state) == :even
            even_state_tot += 1
            push!(even_state, state)
            even_state_num[state] = even_state_tot
        else
            odd_state_tot += 1
            push!(odd_state, state)
            odd_state_num[state] = odd_state_tot
        end
    end
    return Dict{Symbol, Any}(:even_state => even_state, :odd_state => odd_state, :even_state_num => even_state_num, :odd_state_num => odd_state_num, :even_state_tot => even_state_tot, :odd_state_tot => odd_state_tot)
end


parity_states_list (generic function with 1 method)

### Then we can use the above information to construct Hamiltonian for even/odd sectors

In [6]:
state_gen = parity_states_list(L=L)
even_state = state_gen[:even_state]
even_state_num = state_gen[:even_state_num]
odd_state_num  = state_gen[:odd_state_num]
odd_state = state_gen[:odd_state]
even_state_tot = state_gen[:even_state_tot]
odd_state_tot = state_gen[:odd_state_tot]

UndefVarError: [91mUndefVarError: L not defined[39m

In [7]:
function update_val(row_inds, col_inds, vals;row_ind, col_ind, val)
    push!(row_inds, row_ind)
    push!(col_inds, col_ind)
    push!(vals, val)
end

function Hamiltonian1(;N::Int64=2, J1=1, J2=1, h=1, nearest_neib_list, second_neib_list)
    row_inds = Int64[]
    col_inds = Int64[]
    vals = Float64[]
    for state in 0:(2^N-1) #loop over all states
        state_binary = digits!(zeros(Int64, 64), state, base = 2)
        for i in 1:N #loop over all sites in a given state
            flipped_state = state ⊻ (1<<(i-1))
            update_val(row_inds, col_inds, vals, row_ind=state+1, col_ind = flipped_state+1, val = (1/2)*h)
            for j in nearest_neib_list[i] #loop over(compare) all neighbors of a given site
                update_val(row_inds, col_inds, vals, row_ind = state+1, col_ind = state+1, val = (state_binary[i]-1/2)*(state_binary[j]-1/2)*J1/2)
            end
            for j in second_neib_list[i] #loop over(compare) all neighbors of a given site
                update_val(row_inds, col_inds, vals, row_ind = state+1, col_ind = state+1, val = (state_binary[i]-1/2)*(state_binary[j]-1/2)*J2/2)
            end
        end
    end
    return sparse(row_inds, col_inds, vals, 2^N, 2^N, +)
end

function Hamiltonian2(;N::Int64=2, J1=1, J2=1, h=1, nearest_neib_list, second_neib_list)
    row_inds = Int64[]
    col_inds = Int64[]
    vals = Float64[]
    for state in 0:(2^N-1) #loop over all states
        state_binary = digits!(zeros(Int64, 64), state, base = 2)
        for i in 1:N #loop over all sites in a given state
            if state_binary[i] == 1
                update_val(row_inds, col_inds, vals, row_ind=state+1, col_ind = state+1, val = - h/2)
            else
                update_val(row_inds, col_inds, vals, row_ind=state+1, col_ind = state+1, val =  h/2)
            end
            for j in nearest_neib_list[i] #loop over(compare) all neighbors of a given site
                flipped_state = state ⊻ (1<<(i-1))
                flipped_state = flipped_state ⊻ (1<<(j-1))
                update_val(row_inds, col_inds, vals, row_ind=state+1, col_ind = flipped_state+1, val =  (1/4)*(J1/2))
            end
            for j in second_neib_list[i] #loop over(compare) all neighbors of a given site
                flipped_state = state ⊻ (1<<(i-1))
                flipped_state = flipped_state ⊻ (1<<(j-1))
                update_val(row_inds, col_inds, vals, row_ind=state+1, col_ind = flipped_state+1, val =  (1/4)*(J2/2))
            end
        end
    end
    return sparse(row_inds, col_inds, vals, 2^N, 2^N, +)
end

@time H1 = Hamiltonian1(;N=N, J1=J1, J2=J2, h=h, nearest_neib_list=nearest_neib_list, second_neib_list=second_neib_list)
@time H2 = Hamiltonian2(;N=N, J1=J1, J2=J2, h=h, nearest_neib_list=nearest_neib_list, second_neib_list=second_neib_list)

  0.618498 seconds (799.38 k allocations: 401.212 MiB, 13.54% gc time)
  0.454744 seconds (178.18 k allocations: 402.291 MiB, 13.44% gc time)


65536×65536 SparseMatrixCSC{Float64,Int64} with 3211264 stored entries:
  [1    ,     1]  =  0.8
  [4    ,     1]  =  0.25
  [7    ,     1]  =  0.25
  [10   ,     1]  =  0.25
  [13   ,     1]  =  0.25
  [18   ,     1]  =  0.25
  [25   ,     1]  =  0.25
  [35   ,     1]  =  0.25
  [37   ,     1]  =  0.25
  [49   ,     1]  =  0.25
  [67   ,     1]  =  0.25
  [69   ,     1]  =  0.25
  ⋮
  [65440, 65536]  =  0.25
  [65468, 65536]  =  0.25
  [65470, 65536]  =  0.25
  [65488, 65536]  =  0.25
  [65500, 65536]  =  0.25
  [65502, 65536]  =  0.25
  [65512, 65536]  =  0.25
  [65519, 65536]  =  0.25
  [65524, 65536]  =  0.25
  [65527, 65536]  =  0.25
  [65530, 65536]  =  0.25
  [65533, 65536]  =  0.25
  [65536, 65536]  =  -0.8

## Using Lanczos to calculate the ground state and compare see if the Hamiltonians are compatible

I'm using ```KrylovKit.jl``` to calculate.

It appears that the eigenstates is automatically normalized by the package.

In [8]:
eigs1 = eigsolve(H1, 1, :SR, eltype(H1), krylovdim=100)
eigs2 = eigsolve(H2, 1, :SR, eltype(H2), tol = 10^(-20))
eigstate1 = eigs1[2][1]
eigstate2 = eigs2[2][1]

65536-element Array{Float64,1}:
  0.016817242017314996  
  1.3930094714471133e-15
  4.839487804443511e-15 
 -0.006785648867551298  
  9.716991952177162e-15 
  0.0022013510192278973 
 -0.006785648867535335  
 -7.846184059086592e-16 
 -2.7073523668353646e-15
 -0.006785648867536057  
  0.0022013510192268678 
  1.4486132997679351e-15
 -0.00678564886754581   
  ⋮                     
  3.9894024595021e-15   
  0.007486559208545635  
 -0.022953490246453337  
 -1.1172187060476655e-14
 -2.343845774870522e-15 
 -0.022953490246450776  
  0.007486559208549025  
  3.9818321708968935e-14
 -0.02295349024650408   
  1.988600730404991e-14 
  5.941864528869566e-15 
  0.08496250704939282   

In [9]:
eigs1

([-4.041922731843299, -4.041069018081999, -4.040867225724781, -4.040492295350271], Array{Float64,1}[[-2.152909254009893e-11, 4.3170853346137224e-10, 4.3170851702962154e-10, -4.1969564544966966e-9, 4.317085406599208e-10, -9.711433574773234e-9, -4.196956480751543e-9, 4.159796192063053e-8, 4.317085105298895e-10, -4.196956365219907e-9  …  -4.196956398506506e-9, 4.317084837042077e-10, 4.159796192640005e-8, -4.196956470527898e-9, -9.711433581920562e-9, 4.3170850941093654e-10, -4.1969563773178205e-9, 4.3170849837145285e-10, 4.317085083688083e-10, -2.152920087060539e-11], [1.5823449809717596e-17, -1.587232738038857e-10, -6.711474893879193e-11, 3.2387107847885945e-9, -4.471388097269126e-11, 4.18860024427593e-9, 1.6037182572505857e-9, -4.1335851799222724e-8, 7.111868093108892e-11, 1.2563246538901362e-9  …  -1.256324662971333e-9, -7.111872423549121e-11, 4.13358517774502e-8, -1.6037181919584102e-9, -4.188600238781877e-9, 4.4713828570682276e-11, -3.2387108093298947e-9, 6.71146908217823e-11, 1.58723

## Converting states from the basis of H1 to the basis of H2


# Now we could calculate quantities given that we have the ground state.

Here x means the direction of h, y means the direction of coupling.

- $m=\frac{1}{N}\sum_{i} \langle \sigma_i^x \rangle$

Calculating this using H2

- $S_{\pi}=\frac{1}{N}\sum_{i,j}\langle\sigma_i^z\sigma_j^z\rangle (-1)^{i+j}$

Calculating this using H1

- $F_{cl}=\frac{1}{N_{pl}} \sum_{p_1,p_2} \langle f_{p_1} f_{p_2}\rangle$

Calculating this using H1

- $F_{QM}=\frac{1}{N_{pl}} \sum_{p_1,p_2} \langle o_{p_1} o_{p_2}\rangle$ with $o_p = \sigma_1^+\sigma_2^-\sigma_3^+\sigma_4^- + \sigma_1^-\sigma_2^+\sigma_3^-\sigma_4^+$

Calculating this using H1? I think

- $S_A = -Tr \rho_A \log{\rho_A}$

Where $\rho_A = Tr_B \rho$

Calculating this using either

- $Fidelity = 2\cdot\frac{1 - |\langle \psi_0(h)|\psi_0(h+\delta h)\rangle|}{\delta h^2} $

Calculating this using either

## Measuring m using H2

We basically just need to construct matrix of $m$ in the basis out which you constructed the hamiltonian H2.

In H2, $m$ is the magnetization along z direction. So:

$m=\frac{1}{N}\sum_{i} \langle \sigma_i^z \rangle$

In [53]:
function m_H2(state::Array{Float64,1}; N::Int64)
    m = spzeros(2^N,2^N)
    for basis_state in 0:(2^N-1) #loop over all states
        basis_state_binary = digits!(zeros(Int64, 64), basis_state, base = 2)
        # calculating total spin along z direction, considering it's spin 1/2
        m[basis_state+1, basis_state+1] += (sum(basis_state_binary)-1/2*N)/N
    end
    #now that we have matrix m, calculate the average m_val:
    m_val = conj.(state')*m*state
    return m_val[1] #taking the 1st value of m_val because it's recognized as a length 1 Array
end

@time m_H2(eigstate2; N=N)

  0.318501 seconds (131.22 k allocations: 45.693 MiB)


0.025195475951206716

## Measuring M with faster speed

In [11]:
# new M that should be faster
function m_H2(state::Array{Float64,1}; N::Int64)
    m = 0
    for basis_state in 0:(2^N-1) #loop over all states
        basis_state_binary = digits!(zeros(Int64, 64), basis_state, base = 2)
        # calculating total spin along z direction, considering it's spin 1/2
        m += conj(state[basis_state+1])*state[basis_state+1]*(sum(basis_state_binary)-1/2*N)/N
    end
    return m
end

@time m_H2(eigstate2; N=N)

  0.044646 seconds (110.79 k allocations: 41.419 MiB)


0.05489366313948077

## Measuring $S_{\pi}$ using H1

Constructing $S_{\pi}$ matrix using basis of $H1$

$S_{\pi}=\frac{1}{N}\sum_{i,j}\langle\sigma_i^z\sigma_j^z\rangle (-1)^{i+j}$

In [55]:
function S_pi_H1(state::Array{Float64,1}; N::Int64, L1::Int64, L2::Int64)
    S_pi = spzeros(2^N,2^N)
    for basis_state in 0:(2^N-1) #loop over all states
        basis_state_binary = digits!(zeros(Int64, 64), basis_state, base = 2)
        for i in 1:N #loop over all sites in a given state
            for j in 1:N #loop over all sites again
                 S_pi[basis_state+1,basis_state+1] += (basis_state_binary[i]-1/2)*(basis_state_binary[j]-1/2)*(-1)^(sum(coordinate(i;L1=L1, L2=L2))+sum(coordinate(j;L1=L1, L2=L2)))/N
            end
        end
    end
    #now that we have matrix m, calculate the average m_val:
    S_pi_val = conj.(state')*S_pi*state
    return S_pi_val[1] #taking the 1st value of m_val because it's recognized as a length 1 Array
end

@time S_pi_H1(eigstate1;N=N, L1=L1, L2=L2)

  0.957318 seconds (157.64 k allocations: 47.075 MiB)


0.0006311168096688083

## Measuring $S_{\pi}$ using H1 with faster speed

In [12]:
function S_pi_H1(state::Array{Float64,1}; N::Int64, L1::Int64, L2::Int64)
    #build up the phase factor for use later
    phase_factor = zeros(Int64, N, N)
    for i in 1:N #loop over all sites in a given state
        for j in 1:N #loop over all sites again
            phase_factor[i, j] = (-1)^(sum(coordinate(i;L1=L1, L2=L2))+sum(coordinate(j;L1=L1, L2=L2)))
        end
    end
    # start calculating S_pi
    S_pi = 0
    for basis_state in 0:(2^N-1) #loop over all states
        basis_state_binary = digits!(zeros(Int64, 64), basis_state, base = 2)
        for i in 1:N #loop over all sites in a given state
            for j in 1:N #loop over all sites again
                 S_pi += conj(state[basis_state+1])*state[basis_state+1]*(basis_state_binary[i]-1/2)*(basis_state_binary[j]-1/2)*phase_factor[i, j]/N
            end
        end
    end
    return S_pi #taking the 1st value of m_val because it's recognized as a length 1 Array
end

@time S_pi_H1(eigstate1;N=N, L1=L1, L2=L2)

  0.107493 seconds (157.05 k allocations: 43.782 MiB)


0.44220046069513136

In [13]:
function S_entangle(state::Array{Float64,1}; N::Int64, L1::Int64, L2::Int64, second_neib_list)
    #find the first four spin interacting plaquette
    plaq_sites = Int64[] #Has to be an array to make it ordered
    for site in 1:N #loop over all sites
        site_coord = coordinate(site,L1=L1, L2=L2)
        if bit_pos((mod1(site_coord[1]-1, L2),mod1(site_coord[2]-1, L1)), L1=L1, L2=L2) ∈ second_neib_list[site]
            push!(plaq_sites, site)
            push!(plaq_sites, bit_pos((mod1(site_coord[1]+1, L2),site_coord[2]), L1=L1, L2=L2))
            push!(plaq_sites, bit_pos((mod1(site_coord[1]+1, L2),mod1(site_coord[2]+1, L1)), L1=L1, L2=L2))
            push!(plaq_sites, bit_pos((site_coord[1],mod1(site_coord[2]+1, L1)), L1=L1, L2=L2))
            break
        end
    end
    #println(plaq_sites)
    env_sites = Int64[]
    for site in 1:N
        if site ∉ plaq_sites
            push!(env_sites, site)
        end
    end
    #println(env_sites)
    C = zeros(Number, 16, Int(2^N/16))
    for basis_state in 0:(2^N - 1)
        basis_state_binary = digits!(zeros(Int64, 64), basis_state, base = 2)
        a = 1; b = 1 # start at 1 to avoid 0 as numeration number
        for i in 1:4
            a += basis_state_binary[plaq_sites[i]] * 2^(i-1)
        end
        for i in 1:(N-4)
            b += basis_state_binary[env_sites[i]] * 2^(i-1)
        end
        C[a, b] = state[basis_state + 1]        
    end
    Sing_vals = svd(C).S
    return  -sum(Sing_vals.^2 .* log.(Sing_vals.^2))
end

@time S_entangle(eigstate1;N=N, L1=L1, L2=L2, second_neib_list=second_neib_list)

  0.246033 seconds (1.09 M allocations: 87.819 MiB, 8.90% gc time)


1.7896261747186402

## Below we write a driver  to calculate m and $S_{\pi}$ for different h

Calculate Fidelity in the driver

In [16]:
function driver(;L1::Int64, L2::Int64, h_vals, J2)
    N = L1*L2
    J1 = 1
    nearest_neib_list = nearest_neib_list_gen(L1=L1, L2=L2)
    second_neib_list = second_neib_list_gen(L1=L1, L2=L2)
    #plaquette_list = plaquette_list_gen(;L=L, neib_list=neib_list)
    m = []
    S_pi = []
    Fidelity = []
    #Fcl_vals = []
    #Fqm_vals = []
    S_entangle_vals = []
    eigstate_prev = zeros(Float64, 2^N)
    h_prev = -1
    for i in 1:length(h_vals)
        h = h_vals[i]
        H1 = Hamiltonian1(;N=N, J1=J1, J2=J2, h=h, nearest_neib_list=nearest_neib_list, second_neib_list=second_neib_list)
        eigstate1 = eigsolve(H1, 1, :SR, eltype(H1), krylovdim = 100)[2][1]
        #calculating Fidelity using eigenstates of H1
        Fid = conj.(eigstate_prev')*eigstate1
        (i == 1) && (Fid[1] = 1) #set Fid to be 1 manually for the first h
        push!(S_pi, S_pi_H1(eigstate1;N=N, L1=L1, L2=L2))
        push!(Fidelity, 2*(1-abs(Fid[1]))/(h-h_prev)^2)
        #push!(Fcl_vals, Fcl(eigstate1; L=L, neib_list=neib_list, plaquette_list=plaquette_list))
        #push!(Fqm_vals, Fqm(eigstate1; L=L, neib_list=neib_list, plaquette_list=plaquette_list))
        push!(S_entangle_vals, S_entangle(eigstate1;N=N, L1=L1, L2=L2, second_neib_list=second_neib_list))
        eigstate_prev = eigstate1
        
        #Start Calculating using H2
        H2 = Hamiltonian2(;N=N, J1=J1, J2=J2, h=h, nearest_neib_list=nearest_neib_list, second_neib_list=second_neib_list)
        eigstate2 = eigsolve(H2, 1, :SR, eltype(H2), krylovdim = 100)[2][1]
        push!(m, m_H2(eigstate2; N=N))
        h_prev = h
    end
    #return (h_vals, m, S_pi, Fidelity, Fcl_vals, Fqm_vals, S_entangle_vals)
    return (h_vals, J2, m, S_pi, Fidelity, S_entangle_vals)
end

L1=4; L2=4; h_vals = range(0.01, 0.5, length = 50); J2_vals = [1]
for J2 in J2_vals
    @time result = driver(L1=L1, L2=L2, h_vals=h_vals, J2=J2)

    my_time = Dates.now()

    time_finished = "Date_$(Dates.format(my_time, "e_dd_u_yyyy_HH_MM_SS"))"
    content = "Square_Spin_Ice_Measurement"
    save_path = "E:/UC Davis/Research/Square Spin Ice/Square-Spin-Ice/Yutan_code/Results/"
    #"/nfs/home/zyt329/Research/Square_spin_ice/result/"
    save_name = save_path*content*"_J2=$(J2)_L1=$(L1)_L2=$(L2)_hmin=$(h_vals[1])_hmax=$(h_vals[end])_"*time_finished*".jld"

    save(save_name, "result", result)
    println("J2 = $(J2) finished")
end

235.283760 seconds (23.29 M allocations: 58.621 GiB, 4.49% gc time)
J2 = 1 finished
